# **N1. Star detection**

Notebook focused on star detection in a Metis FITS image.

**Load libraries**


In [4]:
import starfunctions as sf
import matplotlib.pyplot as plt
import os

## 1. Introduction

Once all the dataset is downloaded, the next step is to perform star detection. For the first phase, just visible light range images will be used. Then, an algorithm will be used for retrieve the stars location given.

### 1.1 Pipeline for star detection
As the main object is to detect objects in the images, first we need to extract all the objects that are stars. In order to do this, an algorithm was develop by **Paolo Chioetto**, and the repository can be found in the following link:

    https://github.com/chioettop/metis_star_utils

**Metis_star_utils**

Simulates Metis's Field-of-view using the *SO SPICE KERNEL*. Then it takes as input three parameters:
- **Timestamp**: date and time from where it is retrieved the real-time coordinates of metis.
- **Bandwidth**: bandwith used for detecting stars (Visible-light or UltraViolet)
- **StarMagnitude**: maximum star apparent magnitude able to detect.
- Another important parameters are used such as: sun radius, id (solar orbiter), metis fov.

With this information, timestamp is converted into **ephimeris time** and these angles from the metis are retrieved:
1. Right Ascencion (RA): Angular distance eastward along the celestial equator (longitude).
2. Declination (DEC): Angular distance from north to south (latitude).
3. Roll (ROLL) : Rotation of the instrument along the principal axis (boresight).

Once the angles are extracted, using the SIMBAD (Set of Identifications, Measurements, and Bibliography for Astronomical Dataset) Catalogue, it is queried in order to search for stars up to the desired StarMagnitude. Stars located in the inner metis FOV are discarded as well as stars in the outer FOV. Later an image will be displayed for better understanding.

Finally, it returns WCS (world coordinate system) from the previously computed angles and convert stars position into images pixels.

### 1.2 Implemented algorithm for star detection
The mentioned pipeline is implemented in the following function:

In [5]:
def stars_detector(kernel, path, time, UV=False):
    """
    Retrieve a dataframe with stars on a given time coordinate according to Metis.

    Parameters:
        kernel (str): SPICE kernel.
        path(str): path to SPICE kernels.
        time (str): time in SCS.
        UV (bool): specified bandwidth. If false, visible-light bandwidth is chosen.

    Returns:
        catalog_stars (table): astropy table with detected stars.
        -wcs.wcs.cdelt[0] (float): pixel scale.
        wcs.wcs.crpix (tuple): center of image.
    """

    try:
        # Load kernel.
        spice = sf.load_kernel(kernel, path)

        # Convert time to ephemeris time.
        et = sf.scs2et(time, path)

        # Calculate boresight orientation.
        ra, dec, roll = sf.boresight(et, path, UV)
        wcs = sf.wcs_from_boresight(ra, dec, roll, UV)

        # Query stars from Simbad catalog.
        catalog_stars = sf.simbad_search(ra, dec, max_mag=6)
        if catalog_stars is None or len(catalog_stars) == 0:
            return None

        # Transform star coordinates to sensor coordinates.
        x, y = wcs.wcs_world2pix(catalog_stars["ra"], catalog_stars["dec"], 0)

        # Add sensor coordinates to the DataFrame.
        catalog_stars["xsensor"] = x
        catalog_stars["ysensor"] = y

        # Filter stars outside the sensor bounds.
        in_bounds = (x >= 0) & (y >= 0) & (x <= wcs.pixel_shape[1]) & (y <= wcs.pixel_shape[0])
        catalog_stars = catalog_stars[in_bounds]
        if len(catalog_stars) == 0:
            return None

        return catalog_stars, -wcs.wcs.cdelt[0], wcs.wcs.crpix

    except Exception as e:
        print(f"Error: {e}")
        return None

    finally:
        sf.unload_kernel(kernel, path)

Where the functions can be found in the **starfunctions.py**

## 2. Star detection

Now, let's make a simple example using the L0 FITS image from the last notebook and search for the possible stars.

In [ ]:
# Input parameters.
KERNEL_PATH = "./solar-orbiter/kernels/mk/"
KERNEL_NAME = "solo_ANC_soc-flown-mk.tm"
